# HoSt Predictive Model

In [1]:
import sys

assert sys.version_info >= (3, 7)

In [5]:
from packaging import version
import sklearn

assert version.parse(sklearn.__version__) >= version.parse("1.0.1")

In [6]:
import matplotlib.pyplot as plt

plt.rc('font', size=12)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=12)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

In [7]:
import numpy as np

np.random.seed(42)

In [3]:
import sys
# 1) Upgrade packaging tools  
!{sys.executable} -m pip install --upgrade pip setuptools wheel

# 2) Install NumPy headers and Cython for builds  
!{sys.executable} -m pip install numpy Cython

# 3) Install pmdarima (required by TBATS)  
!{sys.executable} -m pip install pmdarima

# 4) Finally install TBATS  
!{sys.executable} -m pip install tbats








Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached pmdarima-2.0.4.tar.gz (630 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build pmdarima


  error: subprocess-exited-with-error
  
  × Building wheel for pmdarima (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [43 lines of output]
      Partial import of pmdarima during the build process.
      
      Requirements: ['joblib>=0.11\nCython>=0.29,!=0.29.18,!=0.29.31\nnumpy>=1.21.2\npandas>=0.19\nscikit-learn>=0.22\nscipy>=1.3.2\nstatsmodels>=0.13.2\nurllib3\nsetuptools>=38.6.0,!=50.0.0\npackaging>=17.1  # Bundled with setuptools, but want to be explicit\n']
      Adding extra setuptools args
      Setting up with setuptools
      Traceback (most recent call last):
        File "C:\Users\amirg\AppData\Roaming\Python\Python313\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 389, in <module>
          main()
          ~~~~^^
        File "C:\Users\amirg\AppData\Roaming\Python\Python313\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 373, in main
          json_out["return_val"] = hook(**hook_input["kwargs"

Defaulting to user installation because normal site-packages is not writeable
  Using cached tbats-1.1.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached pmdarima-2.0.4.tar.gz (630 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Using cached tbats-1.1.3-py3-none-any.whl (44 kB)
Failed to build pmdarima


  error: subprocess-exited-with-error
  
  × Building wheel for pmdarima (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [43 lines of output]
      Partial import of pmdarima during the build process.
      
      Requirements: ['joblib>=0.11\nCython>=0.29,!=0.29.18,!=0.29.31\nnumpy>=1.21.2\npandas>=0.19\nscikit-learn>=0.22\nscipy>=1.3.2\nstatsmodels>=0.13.2\nurllib3\nsetuptools>=38.6.0,!=50.0.0\npackaging>=17.1  # Bundled with setuptools, but want to be explicit\n']
      Adding extra setuptools args
      Setting up with setuptools
      Traceback (most recent call last):
        File "C:\Users\amirg\AppData\Roaming\Python\Python313\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 389, in <module>
          main()
          ~~~~^^
        File "C:\Users\amirg\AppData\Roaming\Python\Python313\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 373, in main
          json_out["return_val"] = hook(**hook_input["kwargs"

In [4]:
# --------------------------------------
# Complete drop-in replacement (copy-paste)
# Forecasts hourly prices for all of 2024
# using a SARIMAX seasonal ARIMA model
# --------------------------------------
# If statsmodels is missing, run:
#   pip install statsmodels

import pandas as pd
import matplotlib.pyplot as plt
from tbats import TBATS

# 1) Load and clean (same as before)
df = (
    pd.read_csv("NetherlandsTrain.csv", parse_dates=["Datetime (UTC)"])
      .sort_values("Datetime (UTC)")
      .dropna(subset=["Price (EUR/MWhe)"])
      .set_index("Datetime (UTC)")
      .asfreq("h")
)
y = df["Price (EUR/MWhe)"]


ModuleNotFoundError: No module named 'tbats'

In [ ]:
# 2) Specify and fit TBATS with multiple seasonal periods:
#    daily=24h, weekly=168h, yearly≈8760h (leap-year ignored in fit)
estimator = TBATS(
    seasonal_periods=(24, 168, 8760),
    use_arma_errors=True,    # include ARMA errors on the residuals
    n_jobs=1                 # set >1 to parallelize fitting
)
model = estimator.fit(y)

15:26:23 - cmdstanpy - INFO - Chain [1] start processing
15:27:23 - cmdstanpy - INFO - Chain [1] done processing


In [ ]:
# 3) Forecast every hour of 2024
n_hours_2024 = 24 * 366    # 2024 is a leap year
forecast_values = model.forecast(steps=n_hours_2024)

# Build a DatetimeIndex for 2024
idx_2024 = pd.date_range(
    start="2024-01-01 00:00:00",
    periods=n_hours_2024,
    freq="H",
    tz=None
)

forecast_2024 = pd.Series(forecast_values, index=idx_2024)

In [ ]:
# 4) Plot history + TBATS forecast
plt.figure(figsize=(14, 6))
plt.scatter(y.index, y.values,
            s=10, alpha=0.5, label="Historical price")
plt.plot(forecast_2024.index, forecast_2024.values,
         color="green", linewidth=1.5,
         label="TBATS forecast 2024")

plt.xlabel("Datetime (UTC)")
plt.ylabel("Price (EUR/MWhe)")
plt.title("Dutch electricity price • history and TBATS forecast for 2024")
plt.legend()
plt.tight_layout()
plt.gcf().autofmt_xdate()
plt.show()

Forecasted price at 2024-12-30 20:00:00 is -123.87 EUR/MWhe


In [ ]:
# ── 5. Peek at the first 11 hourly forecasts of 2024 ─────────
pred_slice = pred_2024.loc["2024-01-01 01:00:00" : "2024-01-01 11:00:00"]
print(pred_slice)

pred_slice = pred_2024.loc["2024-07-01 01:00:00" : "2024-07-01 11:00:00"]
print(pred_slice)

Series([], Freq: h, Name: pred, dtype: float64)
Series([], Freq: h, Name: pred, dtype: float64)
